Capture image from screen on countdown of 10seconds when pressed specific key

In [1]:
import cv2
import time

def capture_image_with_countdown(camera_index=0, time_limit=10):
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    countdown_started = False
    
    start_time = 0 # 

    rect_x, rect_y, rect_w, rect_h = 100, 100, 200, 200
    rect_color = (0, 255, 0)  # Green color

    while True:
        ret, frame = cap.read()

        key = cv2.waitKey(1)

        if key == ord(' '): 
            countdown_started = True
            start_time = time.time()

        # Check if countdown has started
        if countdown_started:
            # Calculation of remaining time
            elapsed_time = time.time() - start_time
            remaining_time = max(0, time_limit - elapsed_time)

            # for count down here
            cv2.putText(frame, f"Time left: {int(remaining_time)} seconds", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Draw the green rectangle
            cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_w, rect_y + rect_h), rect_color, 2)

        # Display the frame 
        cv2.imshow('Camera Feed', frame)

        # Check if the time limit has been reached
        if countdown_started and elapsed_time >= time_limit:
            break
        
        if countdown_started and key == 13:
            roi = frame[rect_y:rect_y + rect_h, rect_x:rect_x + rect_w]

            # Save the captured image
            cv2.imwrite('captured_image.jpg', roi)
            print("Image captured successfully.")
            break

        if key == 27:  # Press 'Esc' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

camera_index = 0    #going to change this if we are using external camera
time_limit = 10
capture_image_with_countdown(camera_index, time_limit)


Passing the detected images here and processing everything


In [ ]:
import cv2
from ultralytics import YOLO
import time as t
import os

# process_img()
def process_img():
    # Define class labels
    
    class_labels = {
        0: 'athlete-foot', 1: 'cellulitis', 2: 'impetigo', 
        3: 'nail-fungus', 4: 'ringworm', 5: 'cutaneous', 
        6: 'chickenpox', 7: 'shingles'
    }

    # Get the current working directory
    current_folder = os.getcwd()

    # Construct the image path relative to the current working directory
    img_path = os.path.join(current_folder, 'normal_images', 's.webp')

    # Construct the model path relative to the current working directory
    model_path = os.path.join(current_folder, 'weights', 'best.pt')

    # Load the image
    img = cv2.imread(img_path)

    # Load the model
    model = YOLO(model_path)


    # Get predictions
    results = model.predict(source=img, save=True, project=r"C:\Users\Harsh\OneDrive\Desktop\epoch 5k\projects_predictions")
    # print(results)
    # Extract class ids and confidence scores
    if results and results[0].boxes:
        boxes = results[0].boxes
        print(f"Number of classes detected: {len(set(boxes.cls))}")

        for box in boxes:
            class_id = int(box.cls)
            confidence_score = round(float(box.conf), 3)
            class_label = class_labels.get(class_id, f'Class-{class_id}')
            print(f"Class: {class_label}, Confidence Score: {confidence_score}")
            
    else:
        print("No objects detected or no confidence score available.")
        #make filter algorithim here for each and every specific class and keep on passsing till we get confidence score above 70%
        
start_time = t.time()
process_img()
end_time = t.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")